In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
!pip install texthero
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import texthero as hero
from texthero import preprocessing

In [20]:
df = pd.read_csv('/content/gdrive/My Drive/Deakin energy/processed_dataset_IG.csv')
df

,Unnamed: 0,ActionTaken,Address,CauseCommunity,CauseEnvironment,CausePre,CauseTechnical,CauseWorkP,ContactType,CorrectProtection,EventDescription,FailedAssets,FailedExplosion,FailedOilFilled,FailedOtherAssets,IncidentCause,IncidentConsequence,IncidentFireFFactorReportable,IncidentFireSeverity,IncidentLocationType,IncidentType,Lat,Long,MadeSafe,NetworkType,Status,Voltage,WeatherStation,Postcode,Locality,Category
0,0,Crew isolated supply and undertook repairs,"PARA PARK, 1490 HENDY MAIN ROAD, PARAPARAP VIC...",Vehicle,Lightning,Unlikely,Earth fault,Not followed,Phase to earth,2.0,A nearby customer reported sparking of electri...,Conductor (ABC),0,0,0,HVABC cable faulted midspan,Grassfire,1.0,Small: 10 - 1000 sq.m,Roadway,Infrastructure (network-based),-38.288982,144.191502,1,Powercor,Report,22kV AC,Avalon Airport,3240.0,Paraparap,OH Cable
1,1,Crew undertook repairs,"1154-1198 CHRISTIES ROAD, RAVENHALL VIC 3023",Working too close to underground cables,Lightning,Unlikely,Leakage,Not followed,Phase to earth,2.0,A contractor reported that he had contacted an...,No failed asset,0,0,0,Contractor contacted earthing conductor,No Go Zone (contact),1.0,Negligible: no ground fire,Roadway,Infrastructure (network-based),-37.767377,144.738509,1,Powercor,Report,Earthing cable,Laverton Raaf,3023.0,Ravenhall,Dug up
2,2,Crew isolated supply and undertook repairs,"326 DUNBARS ROAD, PETERBOROUGH VIC 3270",Vehicle,Lightning,Unlikely,Corrosion,Not followed,Phase to earth,1.0,A field crew attending an outage found that a ...,Conductor (bare),0,0,0,Conductor broke due to rust,Serious risk to public safety (e.g. live condu...,1.0,Negligible: no ground fire,Agricultural,Infrastructure (network-based),-38.586647,142.901137,1,Powercor,Report,22kV AC,Warrnambool Airport Ndb,3270.0,Peterborough,Connection
3,3,Crew undertook repairs,"41-49 BANK STREET, SOUTH MELBOURNE VIC 3205",Vandalism,Lightning,Unlikely,Leakage,Not followed,Phase to earth,1.0,Interfere and vandalism in substation. Unknown...,Indoor,0,2,0,Vandalism,Loss of supply;Damage to network assets,1.0,Negligible: no ground fire,Roadway,Infrastructure (network-based),-37.832352,144.968452,1,CitiPower,Report,22kV AC,Essendon Airport,3205.0,South Melbourne,Other
4,4,Crew isolated supply and undertook repairs,"199 WILSONS ROAD, WHITTINGTON VIC 3219",Vehicle,Lightning,Unlikely,Leakage,Not followed,Phase to earth,2.0,A nearby customer reported that a high load ha...,Service conductor,0,0,0,Unknown high load contacted LV service cable,No Go Zone (contact);Damage to network assets,1.0,Negligible: no ground fire,Roadway,Infrastructure (network-based),-38.177436,144.390480,1,Powercor,Report,Low voltage AC (<1kV),Avalon Airport,3219.0,Whittington,Vehicle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6438,6499,The crew replaced a 3 phase FOLCB on the pole,"301 ST KILDA STREET, BRIGHTON VIC 3186",Vehicle,Lightning,Unlikely,Earth fault,Not followed,Phase to earth,2.0,A report came in to UE of a FMB sparking at 31...,Connection box,0,0,0,The likely cause was determined by EIO to be a...,Plant/equipment,1.0,Negligible: no ground fire,Residential,Infrastructure (network-based),-37.905307,144.988061,1,United Energy Distribution,Report,Low voltage AC (<1kV),Moorabbin Airport,3186.0,Brighton,Connection
6439,6500,Conductor repaired at point of contact.,"48 Banks Rd, Mount Wallace VIC 3342, Australia",Working too close to overhead lines,Lightning,Unlikely,Leakage,Not followed,Phase to earth,2.0,Report received from a resident to advise that...,Conductor (bare),2,2,2,Crane struck line,No Go Zone (contact);Damage to network assets,1.0,Negligible: no ground fire,Agricultural,Infrastructure (network-based),-37.707726,144.199888,1,Powercor,Report,12.7kV AC (SWER),Avalon Airport,3342.0,Mount Wallace,Vehicle
6440,6501,On arrival crew isolated supply and arranged p...,"196 VALLANCE ROAD, OUYEN VIC 3490",Vehicle,Lightning,Unlikely,Leakage,Not followe

In [21]:
def data(df):
    features = df['EventDescription'] +' ' + df['Address'] + ' '+ df['IncidentCause'] +' ' + df['ActionTaken'] +\
    ' '+ df['FailedAssets'] +' ' + df['Locality']  + ' ' + df['IncidentConsequence'] + ' ' + \
    df['CauseTechnical'] + ' ' + df['CauseTechnical']
    target = df['Category']
    return features, target

features, target = data(df)
features

0       A nearby customer reported sparking of electri...
1       A contractor reported that he had contacted an...
2       A field crew attending an outage found that a ...
3       Interfere and vandalism in substation. Unknown...
4       A nearby customer reported that a high load ha...
                              ...                        
6438    A report came in to UE of a FMB sparking at 31...
6439    Report received from a resident to advise that...
6440    Report received of pole fire at incident locat...
6441    A customer called to report a tractor had hit ...
6442    concrete electrical cover outside of front of ...
Length: 6443, dtype: object

In [22]:
def preprocess_text():
    # cleaning steps
    cleaning_pipeline = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_whitespace,
        preprocessing.remove_punctuation,
        preprocessing.remove_urls,
        preprocessing.remove_brackets,
        preprocessing.remove_stopwords,
        preprocessing.remove_digits,
        preprocessing.remove_angle_brackets,
        preprocessing.remove_curly_brackets,
        preprocessing.stem
    ]

    # apply pipeline to text
    clean_text = features.pipe(hero.clean, cleaning_pipeline)

    return clean_text


clean_text = preprocess_text()
clean_text

0       nearbi custom report spark electr line locat a...
1       contractor report contact earth cabl excav tre...
2       field crew attend outag found 22kv conductor b...
3       interfer vandal substat unknown third parti us...
4       nearbi custom report high load pull wire locat...
                              ...                        
6438    report came ue fmb spark st kilda st brighton ...
6439    report receiv resid advis crane made contact p...
6440    report receiv pole fire incid locat arriv crew...
6441    custom call report tractor hit wire brought gr...
6442    concret electr cover outsid front hous gave cl...
Length: 6443, dtype: object

In [39]:
x_train, x_test, y_train, y_test = train_test_split(clean_text, target, random_state=0, test_size=0.25, shuffle=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4832,), (1611,), (4832,), (1611,))

In [25]:
def tfidf():
    vectorizer = TfidfVectorizer(analyzer='word', max_features=5000)
    vectorizer.fit(clean_text)
    train_tfidf = vectorizer.transform(x_train)
    test_tfidf = vectorizer.transform(x_test)
    return train_tfidf, test_tfidf

train_tfidf, test_tfidf = tfidf()
train_tfidf.shape, test_tfidf.shape

((4832, 5000), (1611, 5000))

In [26]:
import numpy as np
oversample = SMOTE(random_state=0,n_jobs=-1,k_neighbors=5)
train_tfidf, y_train = oversample.fit_resample(train_tfidf, y_train)
print(f'Shape: {train_tfidf.shape}')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



Shape: (14820, 5000)


In [27]:
lrRidge = LogisticRegression(penalty='l2', tol=0.001)
lrRidge.fit(train_tfidf, y_train)
print(accuracy_score(y_test, lrRidge.predict(test_tfidf)))
print(classification_report(y_test, lrRidge.predict(test_tfidf)))

0.8249534450651769
              precision    recall  f1-score   support

    AF Other       0.52      0.63      0.57        93
      Animal       0.92      0.85      0.88        67
   Conductor       0.37      0.65      0.47        34
  Connection       0.80      0.75      0.78       210
    Crossarm       0.88      0.90      0.89       130
      Dug up       0.92      0.88      0.90        92
        Fuse       0.91      0.80      0.85       107
Installation       0.67      0.75      0.71         8
   Lightning       0.97      0.71      0.82        41
    OH Cable       0.62      0.70      0.66        60
       Other       0.94      0.89      0.91       325
        Pole       0.61      0.65      0.63        83
       Trees       0.95      0.92      0.93       134
    UG Cable       0.17      0.17      0.17         6
     Vehicle       0.92      0.92      0.92       221

    accuracy                           0.82      1611
   macro avg       0.74      0.75      0.74      1611
weighte

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [28]:
lrLasso = LogisticRegression(penalty='l1', tol=0.001, solver='liblinear')
lrLasso.fit(train_tfidf, y_train)
print(accuracy_score(y_test, lrLasso.predict(test_tfidf)))
print(classification_report(y_test, lrLasso.predict(test_tfidf)))

0.819366852886406
              precision    recall  f1-score   support

    AF Other       0.50      0.65      0.57        93
      Animal       0.92      0.85      0.88        67
   Conductor       0.35      0.59      0.44        34
  Connection       0.82      0.74      0.78       210
    Crossarm       0.89      0.90      0.89       130
      Dug up       0.91      0.88      0.90        92
        Fuse       0.88      0.82      0.85       107
Installation       0.62      0.62      0.62         8
   Lightning       0.95      0.85      0.90        41
    OH Cable       0.62      0.68      0.65        60
       Other       0.93      0.87      0.90       325
        Pole       0.62      0.61      0.62        83
       Trees       0.90      0.90      0.90       134
    UG Cable       0.30      0.50      0.37         6
     Vehicle       0.92      0.91      0.91       221

    accuracy                           0.82      1611
   macro avg       0.74      0.76      0.75      1611
weighted

In [30]:
lrElastic = LogisticRegression(penalty='elasticnet', tol=0.001, solver='saga', l1_ratio=0.5)
lrElastic.fit(train_tfidf, y_train)
print(accuracy_score(y_test, lrElastic.predict(test_tfidf)))
print(classification_report(y_test, lrElastic.predict(test_tfidf)))

0.8218497827436375
              precision    recall  f1-score   support

    AF Other       0.49      0.63      0.55        93
      Animal       0.89      0.82      0.85        67
   Conductor       0.39      0.65      0.48        34
  Connection       0.82      0.75      0.78       210
    Crossarm       0.88      0.91      0.89       130
      Dug up       0.90      0.88      0.89        92
        Fuse       0.91      0.81      0.86       107
Installation       0.62      0.62      0.62         8
   Lightning       0.94      0.76      0.84        41
    OH Cable       0.62      0.75      0.68        60
       Other       0.93      0.88      0.90       325
        Pole       0.60      0.63      0.62        83
       Trees       0.94      0.93      0.93       134
    UG Cable       0.33      0.50      0.40         6
     Vehicle       0.93      0.90      0.92       221

    accuracy                           0.82      1611
   macro avg       0.75      0.76      0.75      1611
weighte

In [40]:
count_vect = CountVectorizer(ngram_range=(1,2))
count_vect.fit(clean_text) 

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [41]:
train_cv = count_vect.transform(x_train)
test_cv = count_vect.transform(x_test)

In [43]:
lrRidge.fit(train_cv, y_train)
print(accuracy_score(y_test, lrRidge.predict(test_cv)))
print(classification_report(y_test, lrRidge.predict(test_cv)))

0.8435754189944135
              precision    recall  f1-score   support

    AF Other       0.59      0.65      0.62        93
      Animal       0.97      0.84      0.90        67
   Conductor       0.49      0.53      0.51        34
  Connection       0.78      0.89      0.83       210
    Crossarm       0.89      0.93      0.91       130
      Dug up       0.94      0.90      0.92        92
        Fuse       0.87      0.83      0.85       107
Installation       1.00      0.25      0.40         8
   Lightning       0.97      0.71      0.82        41
    OH Cable       0.74      0.62      0.67        60
       Other       0.90      0.91      0.91       325
        Pole       0.70      0.67      0.69        83
       Trees       0.93      0.93      0.93       134
    UG Cable       0.00      0.00      0.00         6
     Vehicle       0.91      0.91      0.91       221

    accuracy                           0.84      1611
   macro avg       0.78      0.70      0.72      1611
weighte

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [44]:
lrLasso.fit(train_cv, y_train)
print(accuracy_score(y_test, lrLasso.predict(test_cv)))
print(classification_report(y_test, lrLasso.predict(test_cv)))

0.8361266294227188
              precision    recall  f1-score   support

    AF Other       0.55      0.58      0.57        93
      Animal       0.95      0.88      0.91        67
   Conductor       0.46      0.53      0.49        34
  Connection       0.78      0.88      0.83       210
    Crossarm       0.89      0.92      0.91       130
      Dug up       0.93      0.88      0.91        92
        Fuse       0.86      0.81      0.84       107
Installation       0.75      0.38      0.50         8
   Lightning       0.94      0.76      0.84        41
    OH Cable       0.71      0.58      0.64        60
       Other       0.90      0.91      0.91       325
        Pole       0.64      0.60      0.62        83
       Trees       0.93      0.91      0.92       134
    UG Cable       0.00      0.00      0.00         6
     Vehicle       0.92      0.94      0.93       221

    accuracy                           0.84      1611
   macro avg       0.75      0.70      0.72      1611
weighte

In [45]:
lrElastic.fit(train_cv, y_train)
print(accuracy_score(y_test, lrElastic.predict(test_cv)))
print(classification_report(y_test, lrElastic.predict(test_cv)))

0.840471756672874
              precision    recall  f1-score   support

    AF Other       0.59      0.65      0.62        93
      Animal       0.98      0.84      0.90        67
   Conductor       0.49      0.59      0.53        34
  Connection       0.77      0.86      0.82       210
    Crossarm       0.88      0.92      0.90       130
      Dug up       0.94      0.89      0.92        92
        Fuse       0.90      0.83      0.86       107
Installation       1.00      0.25      0.40         8
   Lightning       0.94      0.71      0.81        41
    OH Cable       0.73      0.63      0.68        60
       Other       0.89      0.91      0.90       325
        Pole       0.66      0.64      0.65        83
       Trees       0.93      0.93      0.93       134
    UG Cable       0.00      0.00      0.00         6
     Vehicle       0.92      0.92      0.92       221

    accuracy                           0.84      1611
   macro avg       0.78      0.70      0.72      1611
weighted

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

